SEARCH ENGINE WITH TOOLS AND AGENTS



In [1]:
##ARXIV--Research

#Tools creationg

from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [3]:
#Used inbuilt tool of wikipedia
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [4]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arx=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arx.name

'arxiv'

In [5]:
tools=[wiki,arx]

In [7]:
#Custom tools[RAG TOOL]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [11]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
text_split=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_split.split_documents(docs)
vectordb=FAISS.from_documents(documents,HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2"))
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000026E49311670>, search_kwargs={})

In [12]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool=create_retriever_tool(retriever,"langsmith-search","Search any information about Langsmith page")

retriever_tool.name

'langsmith-search'

In [13]:
tools=[wiki,arx,retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\Aditya Pandey\\Downloads\\LLM Projects\\prenv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='Search any information about Langsmith page', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000026E480DEB60>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS ob

In [14]:
#Running all tools with Agents and LLMs
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()

api_key=os.getenv("GROQ_API_KEY")


llm= ChatGroq(groq_api_key=api_key, model_name="llama-3.1-8b-instant")

In [21]:
#Prompt Template

from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain import hub

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{content}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [22]:
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'content'], input_types={'agent_scratchpad': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag

In [23]:
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'content'], input_types={'agent_scratchpad': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typ

In [25]:
agent_executor.invoke({"content":"Tell me about LangSmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'LangSmith'}`


LangSmith docs - Docs by LangChainSkip to main contentLangGraph Platform is now part of LangSmith. Check out the Changelog for more information.Docs by LangChain home pageLangSmithSearch...⌘KGitHubTry LangSmithTry LangSmithSearch...NavigationLangSmith docsGet startedObservabilityEvaluationPrompt engineeringDeploymentHostingOverviewPlansCreate an account and API keyAccount administrationOverviewSet up a workspaceManage organizations using the APIManage billingSet up resource tagsUser managementReferenceLangSmith Python SDKLangSmith JS/TS SDKLangGraph Python SDKLangGraph JS/TS SDKLangSmith APILangGraph Server APIControl Plane API for LangSmith DeploymentAdditional resourcesReleases & changelogsData managementAuthentication methodsFAQsRegions FAQPricing FAQLangSmith docsCopy pageCopy pageLangSmith provides tools for developing, debugging, and deploying LLM applications.
It helps you trace

{'content': 'Tell me about LangSmith',
 'output': "It appears that the provided text is a snippet from the LangSmith documentation. Based on the information provided, LangSmith is a platform that provides tools for developing, debugging, and deploying LLM (Large Language Model) applications. It offers features such as observability, evaluation, deployment, and hosting, and is framework-agnostic, allowing users to use it with or without LangChain's open-source libraries."}

In [26]:
agent_executor.invoke({"content":"Tell me about Naruto"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Naruto'}`


Page: Naruto
Summary: Naruto is a Japanese manga series written and illustrated by Masashi Kishimoto. It tells the story of Naruto Uzumaki, a young, socially isolated ninja who seeks recognition from his peers and dreams of becoming the Hokage, the lleader of his village. The series explores themes of friendship, love, and the struggle between good and evil. It has been adapted into various forms of media, including anime, movies, and video games.

The Naruto series consists of two parts: Naruto and Naruto: Shippuden. The first part follows Naruto's early days as a genin, a young ninja, and his adventures with his teammates Sakura Haruno and Sasuke Uchiha. The second part, Naruto: Shippuden, takes place two and a half years after the first part and follows Naruto as he deals with the consequences of Sasuke's departure and the rise of a new villain, Pain.

The series has gained a large following 

{'content': 'Tell me about Naruto',
 'output': "leader of his village. The series explores themes of friendship, love, and the struggle between good and evil. It has been adapted into various forms of media, including anime, movies, and video games.\n\nThe Naruto series consists of two parts: Naruto and Naruto: Shippuden. The first part follows Naruto's early days as a genin, a young ninja, and his adventures with his teammates Sakura Haruno and Sasuke Uchiha. The second part, Naruto: Shippuden, takes place two and a half years after the first part and follows Naruto as he deals with the consequences of Sasuke's departure and the rise of a new villain, Pain.\n\nThe series has gained a large following worldwide and is known for its engaging storylines, well-developed characters, and epic battles. It has also been praised for its portrayal of themes such as friendship, perseverance, and the importance of understanding and accepting one's differences.\n\nOverall, Naruto is a popular and i

In [27]:
agent_executor.invoke({"content":"Tell me about the paper 1706.03762"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on cIt appears my response was cut off. The summary provided is for the paper "Attention Is All You Need", with the reference 1706.03762. However, the provided reference does not match the paper summary.

> Finished chain.


{'content': 'Tell me about the paper 1706.03762',
 'output': 'It appears my response was cut off. The summary provided is for the paper "Attention Is All You Need", with the reference 1706.03762. However, the provided reference does not match the paper summary.'}